In [ ]:
import pandas as pd
import IPython.display as display
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os, random
import matplotlib.pyplot as plt
print(tf.__version__)

In [ ]:
# # ========== Download Dataset ==========
# #!pip install -U gdown
# import gdown, zipfile
# file_id = "1eyrNGFlM83pf-TETo30Cvjm7kYuCFAqu" # <-- from your real zip file link
# gdown.download(f"https://drive.google.com/uc?id={file_id}", output="dataset.zip", quiet=False)
# with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
#     zip_ref.extractall("dataset")

In [ ]:
# ==========Dataset Preparation ==========
IMAGE_HEIGHT=128
IMAGE_WIDTH=128
BATCH_SIZE=64
def get_pathframe(path):
    '''
    Get all the images paths and its corresponding labels
    Store them in pandas dataframe
    '''
    filenames = os.listdir(path)
    categories = []
    paths=[]
    for filename in filenames:
        paths.append(path+filename)
        category = filename.split('.')[0]
        if category == 'dog':
            categories.append(1)
        else:
            categories.append(0)
    df= pd.DataFrame({
        'filename': filenames,
        'category': categories,
        'paths':paths
    })
    return df
df=get_pathframe("dataset/dataset/")
df.tail(5)

In [ ]:
# ========== Convert to tensor ==========
def load_and_preprocess_image(path):
    '''
    Load each image and resize it to desired shape
    '''
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMAGE_WIDTH, IMAGE_HEIGHT])
    image /= 255.0 # normalize to [0,1] range
    return image

def convert_to_tensor(df):
    '''
    Convert each data and labels to tensor
    '''
    path_ds = tf.data.Dataset.from_tensor_slices(df['paths'])
    image_ds = path_ds.map(load_and_preprocess_image)
    # onehot_label=tf.one_hot(tf.cast(df['category'], tf.int64),2) if using softmax
    onehot_label=tf.cast(df['category'], tf.int64)
    label_ds = tf.data.Dataset.from_tensor_slices(onehot_label)
    return image_ds,label_ds

X,Y=convert_to_tensor(df)
print("Shape of X in data:", X)
print("Shape of Y in data:", Y)

In [ ]:
#Plot Images
dataset=tf.data.Dataset.zip((X,Y)).shuffle(buffer_size=2000, seed=0)
dataset_train=dataset.take(22500)
dataset_test=dataset.skip(22500)
dataset_train=dataset_train.batch(BATCH_SIZE, drop_remainder=True)
dataset_test=dataset_test.batch(BATCH_SIZE, drop_remainder=True)
dataset_train

def plotimages(imagesls):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for image,ax in zip(imagesls, axes):
        ax.imshow(image)
        ax.axis('off')
        
imagesls=[]
for n, image in enumerate(X.take(5)):
    imagesls.append(image) 
plotimages(imagesls)

In [ ]:
#Model Design
def My_CNNmodel():
    model = tf.keras.models.Sequential()
    model.add(layers.Conv2D(8, (3, 3), padding='same',activation='relu',
    input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Conv2D(16, (3, 3),
    padding='same',activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Conv2D(32, (3, 3),
    padding='same',activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Conv2D(64, (3, 3),
    padding='same',activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    opt=tf.keras.optimizers.Adam(0.001)
    model.compile(
        optimizer=opt,
        loss='binary_crossentropy', # loss='categorical_crossentropy' if softmax
        metrics=['accuracy']
    )
    return model
model=My_CNNmodel()
model.summary()

In [ ]:
#Training the model
#You can adjust the epochs to get better training results, be aware with overfitting
hist=model.fit(dataset_train, epochs=2, validation_data=dataset_test)
#Save trained model
model.save("weights/313605013.keras") #Save the model with your student ID

In [ ]:
#Plot training results
def plot_model_history(model_history, acc='accuracy', val_acc='val_accuracy'):
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    
    # Accuracy plot
    axs[0].plot(range(1, len(model_history.history[acc]) + 1),
    model_history.history[acc])
    axs[0].plot(range(1, len(model_history.history[val_acc]) + 1),
    model_history.history[val_acc])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1, len(model_history.history[acc]) + 1))
    axs[0].legend(['train', 'val'], loc='best')
    
    # Loss plot
    axs[1].plot(range(1, len(model_history.history['loss']) + 1),
    model_history.history['loss'])
    axs[1].plot(range(1, len(model_history.history['val_loss']) + 1),
    model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1, len(model_history.history['loss']) + 1))
    axs[1].legend(['train', 'val'], loc='best')
    plt.tight_layout()
    plt.show()
    
plot_model_history(hist)

In [ ]:
#Evaluate the model
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import numpy as np

# Evaluate the model
loss, accuracy = model.evaluate(dataset_test)
print("Test: accuracy = %f ; loss = %f " % (accuracy, loss))

# Predict values
y_pred = model.predict(dataset_test)
y_p = np.where(y_pred > 0.5, 1, 0) # for binary classification

# Extract ground truth labels
test_data = dataset_test.unbatch()
y_g = []
for image, label in test_data:
    y_g.append(label.numpy())
    
# Convert to flat array if needed
y_g = np.array(y_g).flatten()
y_p = y_p.flatten()

# Compute confusion matrix
confusion_mtx = confusion_matrix(y_g, y_p)

# Plot
f, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01, cmap="Blues",
linecolor="gray", fmt='.1f', ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Generate a classification report
report = classification_report(y_g, y_p, target_names=['0','1'])
print(report)